## Word2vec

参考链接：https://www.cnblogs.com/peghoty/p/3857839.html

- 词向量：

简单来说就是词典$D$中任意词$\omega$，指定一个固定长度的实值向量$v(\omega)\epsilon R^{m}$,
$v(\omega)$就称为$\omega$的词向量，m为词向量的长度，关于词向量的进一步理解将放到下一节来讲解

In [ ]:
词向量一种是One-hot representation就是用一个很长的向量来表示一个词，向量的长度就是词典D的大小N
向量的分量只有一个1，其它全部为0,1的位置对应词在词典中的索引，但这种词向量表示有一些缺点如
容易受到维数灾难的困扰，尤其是将其用于深度学习场景，又如它不能很好地刻画词与词之间的相似性。

另一种词向量是Distributed representation，可以克服One-hot上述缺点，其基本想法是：
通过训练将某种语言中的每一个词隐射成一个固定长度的短向量（这里的短是相对于one-hot的长而言），
所有这些向量构成一个词向量空间，而每一向量则可视为该空间中的一个点，在这个空间上引入“距离”，就可以根据词
之间的距离来判断它们之间的相似性，word2vec中采用的就是这种Distributed Representation的词向量
为什么叫做Distributed presentation：对于one-hot representation向量中只有一个非零向量，相对分散（有点风险平摊的感觉））
把词的信息分布到各个分量中去了；

bn-gram模型思想：它作了一个n-1阶的马尔可夫假设，认为一个词出现的概率与它前面的n-1个词相关即 $p(w_{k} | w_{1}^{k-1}) ==p(w_{k} | w_{k-n+1}^{k-1})$,其主要工作是在语料中统计各种词串出现的次数以及平滑化处理，概率值计算好之后就存储起来，下次需要计算一个句子的概率时，只需要找到相关的概率参数，将它们连乘起来就好了。为什么I love you 的概率可以通过P(I)×P(love|I)×P(you|love)P(I)计算呢？其实这么算只是在某个假设下的近似计算，这个假设就是一个词出现的概率只取决于它前N−1个词。所以在二元模型下，I love you的概率可以那么算。 以此类推，三元模型下，I love you的概率就要这么算了：P(I)×P(love|I)×P(you|I,love)P(I)

## Word2vec两个重要模型

- CBOW  
- Skip-gram

相同点：
都包含三层：输入层，投影层和输出层

不同点：

CBOW是已知当前词$w_{t}$的上下文$w_{t-2}$,$w_{t-1}$,$w_{t+1}$,$w_{t+2}$的前提预测当前词$w_{t}$

Skip-gram是已知当前词$w_{t}$的前提下，预测其上下文$w_{t-2}$,$w_{t-1}$,$w_{t+1}$,$w_{t+2}$



## Huffman树

给定n个权值作为n个叶子节点，构造一棵二叉树，若它的带权路径长度达到最小，则称这样的二叉树为最优二叉树，也称Huffman树

#### Huffman树的构造：

给定n个权值{$\omega_{1}$,$\omega_{2}$,$\omega_{3}$,$\cdots$ ,$\omega_{n}$}做为二叉树的n个叶子节点

- 1、将{$\omega_{1}$,$\omega_{2}$,$\omega_{3}$,$\cdots$ ,$\omega_{n}$}看成是有n棵树的森林
- 2、在森林中选出两个根节点的权值最小的树合并，作为一棵新树的左右子树，且新树的根节点是其左、右子树根节点权值之和
- 3、从森林中删除选取的两棵树，并将新树加入森林
- 4、重复2、3步，直到森林中只剩一棵树为此，该树即为所求的Huffman树

### 基于Hierarchical Softmax的模型

#### CBOW模型

损失函数：
  
$$
\begin{aligned}
    L = \sum_{\omega \epsilon {C}}(\log p(\omega | Context(\omega)))\\
\end{aligned} 
$$

网络结构：

- 1、输入层：包含$Context(\omega)$中2c个词的词向量$v(Context(\omega)1)$,$v(Context(\omega)1)$,$v(Context(\omega)1)$,$v(Context(\omega)1)$
m的含义表示词向量的长度

- 2、投影层：将输入层的2c个向量做求和累加，即$X_{\omega} = \sum_{i=1}^{2c}V(Context(\omega))$

- 3、输出层：输出层对应一棵二叉树，它是以语料中出现的词当叶子节点，以各词在语料中出现的次数当权值够造出来
Huffman树，在这棵Huffman树中叶子节点共N(=|D|)个，分别对应词典D中的词，非叶子节点N-1个（图中标黄色的节点）

神经概率语言模型的网络图同CBOW相比主要有以下三处不同：

1、（从输入层到投影层的操作)前者是拼接，后者是通过累加求和

2、（隐藏层）前者有隐藏层，后者无隐藏层。

3、（输出层）前者是线性结构，后者是树型结构

为什么要这么改：
在神经网络概率语言模型中，模型的大部分计算集中在隐藏层和输出层之间的矩阵向量计算，以及输出层上的
softmax归一化运算，CBOW模型对这些计算复杂度高的地方有针对性的进行了改变，首先去掉了隐藏层，其次
输出层改用了Huffman树，从而为利用Hierarchical Softmax技术奠定了基础。
 ![](./pics/cbow.png)    

#### Skip-gram模型

损失函数：
  
$$
\begin{aligned}
    L = \sum_{\omega \epsilon {C}}(\log p( Context(\omega)|\omega ))\\
\end{aligned} 
$$

我们应该把重点放在$\log p(\omega | Context(\omega))$和$\log p( Context(\omega)|\omega )$的构造上，意识到这点很重要
因为它可以让我们目标明确，心无旁骛，不至于陷于一些繁琐的细节中

### 基于Negative Sample的模型

#### CBOW模型

#### Skip-gram模型

#### 负采用算法